In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))
sys.path.append(os.path.abspath(".."))
print(sys.path)
from sheet_manager import SpreadsheetManager
from dotenv import load_dotenv
import requests
from utilities import current_sg_time
from datetime import datetime

load_dotenv('../.env.dev')

def loop_leave_files(token, url=os.environ.get('LEAVE_FOLDER_URL')):

    header = {
        'Authorization': token,
        'Content-Type': 'application/json'
    }

    drive_url = f"https://graph.microsoft.com/v1.0/drives/{os.environ.get('DRIVE_ID')}/items/"

    print(url)
    response = requests.get(url=url, headers=header)

    # response.raise_for_status()
    if not 200 <= response.status_code < 300:
        print("something went wrong when getting files")
        print(response.text)
        return
    
    for value in response.json()['value']:
        if value['name'].endswith(".xlsx"):
            year = value['name'].split('.')[0]
            year_int = int(year)
            current_year = current_sg_time().year
            if not year_int < current_year:
                new_url = drive_url + value['id'] + '/workbook/worksheets'
                sheets_resp = requests.get(url=new_url, headers=header)
                if not 200 <= sheets_resp.status_code < 300:
                    print("something went wrong when getting sheets")
                    return
                months = []
                for obj in sheets_resp.json()['value']:
                    month = obj['name']
                    month_int = datetime.strptime(month, "%B").month
                    if not month_int < current_sg_time().month:
                        months.append(f"{obj['name']}-{year}")
                return months
        else:
            continue

    return months

['/mnt/c/Users/rachm/OneDrive/Desktop/gos_projects/spacychatbot/project/services/app/azure', '/home/rachtrx/mambaforge/envs/chatbot/lib/python310.zip', '/home/rachtrx/mambaforge/envs/chatbot/lib/python3.10', '/home/rachtrx/mambaforge/envs/chatbot/lib/python3.10/lib-dynload', '', '/home/rachtrx/.local/lib/python3.10/site-packages', '/home/rachtrx/mambaforge/envs/chatbot/lib/python3.10/site-packages', '/mnt/c/Users/rachm/OneDrive/Desktop/gos_projects/spacychatbot/project/services', '/mnt/c/Users/rachm/OneDrive/Desktop/gos_projects/spacychatbot/project/services/app']


In [2]:
with open("../token.txt", 'r') as f:
    token = f.read()

mmyy_arr = loop_leave_files(token=token)
mmyy_arr


https://graph.microsoft.com/v1.0/drives/b!miY83vtC4E6hZVysK2MuIIXS4kkbArdAu4a1mtl-tW_FUmaJ5dR7Sq1_9ZAZ_eG9/items/01JE235TFDKUOGX6FMEBGYVYTJJTKLTGOL/children


['March-2024']

In [3]:
manager = SpreadsheetManager(mmyy_arr[0], token=token, dev=True)
az_df = manager.find_all_dates()
az_df

/mnt/c/Users/rachm/OneDrive/Desktop/gos_projects/spacychatbot/project/services/app/azure/sheet_manager.py:309: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  current_details_df.loc[~empty_date_mask, 'date'] = pd.to_datetime(current_details_df.loc[~empty_date_mask, 'date'], format='%d/%m/%Y').dt.date


,date,name,dept,type
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [4]:
import subprocess

try:
    # Run the system command and capture the output
    result = subprocess.run(['systemctl', 'status', 'postgresql'], 
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE,
                            text=True)
    # Check if PostgreSQL is active
    if 'inactive (dead)' in result.stdout:
        print("PostgreSQL service is inactive.")
    else:
        result = subprocess.run(['systemctl', 'stop', 'postgresql'],
                            stdout=subprocess.PIPE, 
                            stderr=subprocess.PIPE,
                            text=True)
        print("PostgreSQL service stopped.")
    # print(result.stdout)
except Exception as e:
    print(f"An error occurred: {e}")
    

PostgreSQL service is inactive.


In [5]:
import psycopg2

try:
    conn = psycopg2.connect(
        dbname='chatbot',
        user=os.environ.get('SQL_USER'),
        password=os.environ.get('PGPASSWORD'),
        host='localhost',
        port=os.environ.get('SQL_PORT')
    )
    print("Connection to the database was successful")
    cur = conn.cursor()
    cur.execute("SELECT job_user.job_no, date, users.name, dept, type, mc_records.is_cancelled FROM mc_records JOIN job_user ON job_user.job_no = mc_records.job_no JOIN users ON job_user.name = users.name")
    rows = cur.fetchall()
    col_names = [desc[0] for desc in cur.description]
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.commit()
        conn.close()

Connection to the database was successful


In [6]:
import pandas as pd

db_df = pd.DataFrame(rows, columns=col_names)
db_df = db_df.loc[db_df.is_cancelled == False]
db_df.drop(columns = ['job_no', 'is_cancelled'], inplace=True)

In [7]:
az_df

,date,name,dept,type
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [8]:
from utilities import get_latest_date_past_8am

mask = ((az_df["date"] >= get_latest_date_past_8am()) | (az_df.isna().all(axis=1)))

az_df = az_df.loc[mask]

az_df = az_df.astype({"date": str, "name": str, "dept": str, "type": str})
az_df

,date,name,dept,type
0,nan,nan,nan,nan
1,nan,nan,nan,nan
2,nan,nan,nan,nan
3,nan,nan,nan,nan
4,nan,nan,nan,nan
5,nan,nan,nan,nan
6,nan,nan,nan,nan
7,nan,nan,nan,nan
8,nan,nan,nan,nan
9,nan,nan,nan,nan


In [9]:
dates_to_rm = pd.merge(az_df, db_df, how="outer", indicator=True).query('_merge == "left_only"').drop(columns='_merge')
dates_to_update = pd.merge(az_df, db_df, how="outer", indicator=True).query('_merge == "right_only"').drop(columns='_merge')

In [10]:
dates_to_rm

,date,name,dept,type
0,nan,nan,nan,nan
1,nan,nan,nan,nan
2,nan,nan,nan,nan
3,nan,nan,nan,nan
4,nan,nan,nan,nan
5,nan,nan,nan,nan
6,nan,nan,nan,nan
7,nan,nan,nan,nan
8,nan,nan,nan,nan
9,nan,nan,nan,nan


In [11]:
dates_to_update

,date,name,dept,type
10,2024-03-03,ICT Hotline,ICT,Medical
11,2024-03-04,ICT Hotline,ICT,Medical
12,2024-03-05,ICT Hotline,ICT,Medical
13,2024-03-06,ICT Hotline,ICT,Medical
14,2024-03-08,ICT Hotline,ICT,Medical
15,2024-03-07,ICT Hotline,ICT,Medical
16,2024-03-09,ICT Hotline,ICT,Medical
17,2024-03-10,ICT Hotline,ICT,Medical
18,2024-03-11,ICT Hotline,ICT,Medical
19,2024-03-12,ICT Hotline,ICT,Medical


In [12]:
indexes_to_rm = dates_to_rm.index.tolist()
print(f"indexes to remove: {indexes_to_rm}")


indexes to remove: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [13]:
manager.delete_from_excel(indexes_to_rm)

In [14]:
data_to_add = dates_to_update.apply(lambda row: [f"'{row['date'].strftime('%d/%m/%Y')}"] + row[['name', 'dept', 'type']].astype(str).tolist(), axis=1).tolist()
data_to_add

[["'03/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'04/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'05/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'06/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'08/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'07/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'09/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'10/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'11/03/2024", 'ICT Hotline', 'ICT', 'Medical'],
 ["'12/03/2024", 'ICT Hotline', 'ICT', 'Medical']]

In [15]:
manager.upload_data(data_to_add)